### Installation
1. Run the first 2 cells
2. Restart runtime
3. Run the rest of the jupyter notebooks (do not run the first 2 cells again)

In [1]:
#!git clone -b main https://github.com/zcemycl/TF2DeepFloorplan.git
!pip install gdown
!pip install --upgrade --no-cache-dir gdown
!gdown https://drive.google.com/uc?id=1czUSFvk6Z49H-zRikTc67g2HUUz4imON
!unzip log.zip
!rm log.zip

Downloading...
From (original): https://drive.google.com/uc?id=1czUSFvk6Z49H-zRikTc67g2HUUz4imON
From (redirected): https://drive.google.com/uc?id=1czUSFvk6Z49H-zRikTc67g2HUUz4imON&confirm=t&uuid=5445d652-a341-466d-88a3-63929612abc1
To: /content/log.zip
100% 113M/113M [00:00<00:00, 209MB/s]
Archive:  log.zip
replace log/store/checkpoint? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: log/store/checkpoint    
replace log/store/events.out.tfevents.1604013900.yui-Inspiron-7590.21326.5.v2? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: log/store/events.out.tfevents.1604013900.yui-Inspiron-7590.21326.5.v2  
replace log/store/events.out.tfevents.1604014000.yui-Inspiron-7590.21649.5.v2? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: log/store/events.out.tfevents.1604014000.yui-Inspiron-7590.21649.5.v2  
replace log/store/events.out.tfevents.1604014116.yui-Inspiron-7590.22149.5.v2? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: log/store/events.out.tfevents.1604014116.y

In [ ]:
import os
os.kill(os.getpid(), 9)
!pip uninstall -y tensorflow tensorflow-cpu tensorflow-gpu protobuf
!pip install dfp==0.0.6.dev1+gb5860f2 --no-deps
!pip install protobuf==3.20 tensorboard==2.12

In [1]:
# gpu
!cd TF2DeepFloorplan && pip install -e .[tfgpu]
# cpu
#!cd TF2DeepFloorplan && pip install -e .[tfcpu]

Obtaining file:///content/TF2DeepFloorplan
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached pdbpp-0.10.3-py2.py3-none-any.whl.metadata (23 kB)
  Using cached protobuf-3.20.0-py2.py3-none-any.whl.metadata (720 bytes)
  Using cached types_requests-2.32.0.20250306-py3-none-any.whl.metadata (2.3 kB)
  Using cached pytype-2024.10.11-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (10 kB)
  Using cached dynaconf-3.2.10-py2.py3-none-any.whl.metadata (9.1 kB)
INFO: pip is looking at multiple versions of dfp[tfgpu] to determine which version is compatible with other requirements. This could take a while.
ERROR: Ignored the following versions that require a different python version: 2020.10.8 Requires-Python <3.9,>=3.5; 2020.11.12 Requires-Python <3.9,>=3.6; 202

### Main Script

In [ ]:
import tensorflow as tf
import sys
from dfp.net import *
from dfp.data import *
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from argparse import Namespace
import os
import gc
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
from dfp.utils.rgb_ind_convertor import *
from dfp.utils.util import *
from dfp.utils.legend import *
from dfp.utils.settings import *
from dfp.deploy import *
print(tf.test.is_gpu_available())
print(tf.config.list_physical_devices('GPU'))

In [ ]:
img_path = './TF2DeepFloorplan/resources/30939153.jpg'
inp = mpimg.imread(img_path)
args = parse_args("--tomlfile ./TF2DeepFloorplan/docs/notebook.toml".split())
args = overwrite_args_with_toml(args)
args.image = img_path

In [ ]:
result = main(args)

In [ ]:
plt.subplot(1,2,1)
plt.imshow(inp); plt.xticks([]); plt.yticks([]);
plt.subplot(1,2,2)
plt.imshow(result); plt.xticks([]); plt.yticks([]);

## Breakdown of postprocessing (step by step)

In [ ]:
model,img,shp = init(args)
logits_cw,logits_r = predict(model,img,shp)

In [ ]:
logits_r = tf.image.resize(logits_r,shp[:2])
logits_cw = tf.image.resize(logits_cw,shp[:2])
r = convert_one_hot_to_image(logits_r)[0].numpy()
cw = convert_one_hot_to_image(logits_cw)[0].numpy()
plt.subplot(1,2,1)
plt.imshow(r.squeeze()); plt.xticks([]); plt.yticks([]);
plt.subplot(1,2,2)
plt.imshow(cw.squeeze()); plt.xticks([]); plt.yticks([]);

In [ ]:
r_color,cw_color = colorize(r.squeeze(),cw.squeeze())
plt.subplot(1,2,1)
plt.imshow(r_color); plt.xticks([]); plt.yticks([]);
plt.subplot(1,2,2)
plt.imshow(cw_color); plt.xticks([]); plt.yticks([]);

In [ ]:
newr,newcw = post_process(r,cw,shp)
plt.subplot(1,2,1)
plt.imshow(newr.squeeze()); plt.xticks([]); plt.yticks([]);
plt.subplot(1,2,2)
plt.imshow(newcw.squeeze()); plt.xticks([]); plt.yticks([]);

In [ ]:
newr_color,newcw_color = colorize(newr.squeeze(),newcw.squeeze())
plt.subplot(1,2,1)
plt.imshow(newr_color); plt.xticks([]); plt.yticks([]);
plt.subplot(1,2,2)
plt.imshow(newcw_color); plt.xticks([]); plt.yticks([]);

In [ ]:
plt.imshow(newr_color+newcw_color); plt.xticks([]); plt.yticks([]);

In [ ]:
over255 = lambda x: [p/255 for p in x]
colors2 = [over255(rgb) for rgb in list(floorplan_fuse_map.values())]
colors = ["background", "closet", "bathroom",
          "living room\nkitchen\ndining room",
          "bedroom","hall","balcony","not used","not used",
          "door/window","wall"]
f = lambda m,c: plt.plot([],[],marker=m, color=c, ls="none")[0]
handles = [f("s", colors2[i]) for i in range(len(colors))]
labels = colors
legend = plt.legend(handles, labels, loc=3,framealpha=1, frameon=True)

fig  = legend.figure
fig.canvas.draw()
plt.xticks([]); plt.yticks([]);
